In [10]:
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

In [12]:
def load_dict_from_file(filename):
    loaded_dict = {}
    with open(filename, 'r') as f:
        for line in f:
            parts = line.strip().split(':')
            subject = parts[0].strip()
            events = [event.strip('][').split('][') for event in parts[1:]]
            # Split each feature and convert to float
            events = [[float(feature) for feature in event.split(',')] for sublist in events for event in sublist]
            loaded_dict[subject] = events
    return loaded_dict

def extend_list(dict):
    all_subj=[]
    for subject_id in dict.keys():
        all_subj.extend(dict[subject_id])
    return all_subj

def mean_of_lists(input_list):
    return [sum(items) / len(items) for items in zip(*input_list)]

In [15]:
labels = {}
with open('labels.txt', 'r') as f:
    for line in f:
        parts = line.strip().split(':')
        subject = parts[0].strip()
        data = parts[1].strip().split('][')
        data = [x.strip('[').strip(']').split(',') for x in data]
        data = [int(x) for sublist in data for x in sublist]
        labels[subject] = data
    

In [4]:
theta_features=load_dict_from_file('theta_features.txt')
frequency_features=load_dict_from_file('frequency_features.txt')
temporal_features=load_dict_from_file('temporal_features.txt')
all_features=load_dict_from_file('all_features.txt')

In [8]:
def classification(kern,feature, labels):
    num_iterations = 100
    test_size = 0.3  
    train_size= 0.7

    model = SVC(kernel=kern, class_weight='balanced') #weighted SVM with rbf kernel
    scaler = StandardScaler()
    
    mean_scores = []
    std_scores = []
    sensitivities = []
    specificities = []
    balanced_accuracies = []
    coefs=[]
    value= np.array(extend_list(feature))
    label= np.array(extend_list(labels))
    
    for _ in range(num_iterations):
        shuffle_split = StratifiedShuffleSplit(test_size=test_size, train_size=train_size) #Monte-Carlo 100 iterations, 70% train, 30% test
        
        iteration_scores = []

        for train_index, test_index in shuffle_split.split(value,label):
            X_train, X_test = value[train_index], value[test_index]
            y_train, y_test = label[train_index], label[test_index]
            
            X_train_scaled = scaler.fit_transform(X_train) #Feature standardization
            X_test_scaled = scaler.transform(X_test)
            
            model.fit(X_train_scaled, y_train)
            
            score = model.score(X_test_scaled, y_test)
            iteration_scores.append(score)
            
            # Calculate confusion matrix
            y_pred = model.predict(X_test_scaled)
            cm = confusion_matrix(y_test, y_pred)
            tn, fp, fn, tp = cm.ravel()
            
            sensitivity = tp / (tp + fn)
            
            specificity = tn / (tn + fp)
            
            balanced_accuracy = (sensitivity + specificity) / 2
            
            sensitivities.append(sensitivity)
            specificities.append(specificity)
            balanced_accuracies.append(balanced_accuracy)
            if kern=='linear':
                coef = model.coef_[0]
                coefs.append(coef)
            else: 
                continue


        mean_score = np.mean(iteration_scores)
        std_score = np.std(iteration_scores)

        print("Mean score: {:.4f}".format(mean_score))
        print("Std score: {:.4f}".format(std_score))
        print("-------------------------------")

        mean_scores.append(mean_score)
        std_scores.append(std_score)

    overall_mean_score = np.mean(mean_scores)
    overall_std_score = np.mean(std_scores)
    print("Overall Mean score: {:.4f}".format(overall_mean_score))
    print("Overall Std score: {:.4f}".format(overall_std_score))

    mean_sensitivity = np.mean(sensitivities)
    std_sensitivity = np.std(sensitivities)
    mean_specificity = np.mean(specificities)
    std_specificity = np.std(specificities)
    mean_balanced_accuracy = np.mean(balanced_accuracies)
    std_balanced_accuracy = np.std(balanced_accuracies)
    mean_coef= mean_of_lists(coefs)
    print("Mean Sensitivity: {:.4f}".format(mean_sensitivity))
    print("Std Sensitivity: {:.4f}".format(std_sensitivity))
    print("Mean Specificity: {:.4f}".format(mean_specificity))
    print("Std Specificity: {:.4f}".format(std_specificity))
    print("Mean Balanced Accuracy: {:.4f}".format(mean_balanced_accuracy))
    print("Std Balanced Accuracy: {:.4f}".format(std_balanced_accuracy))  
    if kern=='linear':
        for feature, score in enumerate(mean_coef):
            print(f"Feature {feature+1}: {score}")
      

### Temporal

In [17]:
classification('linear',frequency_features,labels)

Mean score: 0.6291
Std score: 0.0448
-------------------------------
Mean score: 0.6399
Std score: 0.0371
-------------------------------
Mean score: 0.6449
Std score: 0.0290
-------------------------------
Mean score: 0.6387
Std score: 0.0313
-------------------------------
Mean score: 0.6404
Std score: 0.0421
-------------------------------
Mean score: 0.6443
Std score: 0.0138
-------------------------------
Mean score: 0.6534
Std score: 0.0402
-------------------------------
Mean score: 0.6372
Std score: 0.0405
-------------------------------
Mean score: 0.6368
Std score: 0.0388
-------------------------------
Mean score: 0.6162
Std score: 0.0384
-------------------------------
Mean score: 0.6347
Std score: 0.0248
-------------------------------
Mean score: 0.6307
Std score: 0.0287
-------------------------------
Mean score: 0.6454
Std score: 0.0475
-------------------------------
Mean score: 0.6145
Std score: 0.0332
-------------------------------
Mean score: 0.6474
Std score: 0.02

### Theta

In [ ]:
classification('rbf', theta_features, labels)

### Frequency

In [ ]:
classification('rbf', frequency_features, labels)

### All

In [ ]:
classification('rbf', all_features, labels)